In [ ]:
import statsmodels
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import os

import statsmodels.api as sm

$$
\newcommand{\cor}{\text{Cor}}
\newcommand{\var}{\text{Var}}
\newcommand{\cov}{\text{Cov}}
\newcommand{E}{\text{E}}
$$

Residuals is what remains to be explained after fitting the regression line. The regression model is $Y_i = \beta_0 + \beta_1 X_i + \epsilon_i$. We estimate the unknown parameters $\beta_0$ and $\beta_1$ as
$$
\begin{eqnarray}
\hat{\beta}_1 &=& \cor(Y, X)\frac{S_X}{S_Y} \\
\hat{\beta}_0 &=& \bar{Y} - \hat{\beta}_1\bar{X}
\end{eqnarray}
$$
We will examine the residuals in Galton's data. The fit in diamond data is quite impressive and may not help understand the idea and importance of residuals. The residuals are defined as 
$$
e_i = Y_i - \hat{Y}_i.
$$
An analysis of the residuals helps us understand why a model does not fit well.

In [ ]:
filename = 'galton.csv'
pathname = os.path.join('.', 'datasets', filename)
if os.path.isfile(pathname):
    df: pd.core.frame.DataFrame = pd.read_csv(pathname)
else:
    assert False

In [ ]:
df.head()

In [ ]:
model = sm.OLS(endog=df['child'], exog=sm.add_constant(df['parent']))
result = model.fit()
result.summary()

In [ ]:
Y_hat: np.array = result.predict()
residual: np.array = df['child'] - Y_hat

In [ ]:
np.mean(residual)

In [ ]:
_ = plt.hist(residual, bins=50)
_ = plt.xlabel('residual')
_ = plt.title('Distribution of residuals')

In [ ]:
plt.scatter(df['parent'], residual, s=2)
plt.xlabel('X')
plt.ylabel('e')
_ = plt.title(r'Residual for each $X$_i')

A few observations:

- The spread in the residuals is not constant across the input. Therefore, the assumption that $\epsilon_i$ are iid $N(0, \sigma^2)$ is not valid.
- There also seems to be an oscillatory pattern to the residuals.

In the next cell, we will look at synthetic data designed to show oscillatory residuals when a linear model is fit.

In [ ]:
X = np.linspace(0, 4, 100)
Y = 0.5 * X + np.sin(X)
model_1 = sm.OLS(Y, sm.add_constant(X))
result_1 = model_1.fit()
_ = plt.scatter(X, result_1.resid, s=2)
_ = plt.xlabel('X')
_ = plt.ylabel('E')
_ = plt.title('Residuals')

In the next synthetic data set, we will deliberately add random noise that does not remain constant.

In [ ]:
n_samples = 100
X = np.linspace(0, 4, n_samples)
Y = 0.5 * X + np.sqrt(X) * np.random.normal(0, 1, n_samples)

In [ ]:
model_2 = sm.OLS(Y, sm.add_constant(X))
result_2 = model_2.fit()
_ = plt.scatter(X, result_2.resid, s=2)
_ = plt.xlabel('X')
_ = plt.ylabel('E')
_ = plt.title('Residuals')

The residuals show a blatant violation of the assumption of uniform variance. The violation is called _heteroscedasticity_ while the adherence is called _homoscedasticity_.

MSE or residuals is 
$$
\frac{1}{N-2}\sum_{i=1}^N (e_i - \bar{e})^2 = \frac{1}{N-2}\sum_{i=1}^N e_i^2,
$$
as $\bar{e} = 0$ for a model with intercept. We confirm that using the code in the next two cells. This is called 'variance of residuals'. It differs slightly from the $\var\;E$, which we compute separately.

In [ ]:
result_2.mse_resid

In [ ]:
np.sum(result_2.resid**2)/(n_samples - 2)

In [ ]:
np.var(result_2.resid)